In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install youtokentome

In [3]:

import youtokentome as yttm
from glob import glob

In [4]:
fs = [f for f in glob('../data/ru/**/**/*.txt')]

In [10]:
txts = []
for fn in glob('../data/ru/**/**/*.txt'):
    with open(fn) as f:
        txt = f.read()
        txts.append(txt)

traintxt = '\n'.join(txts)

In [12]:
import re
words = re.split(r'\W+', traintxt)

In [14]:
lowt = ' '.join(words)
lowt = lowt.lower()

In [17]:
train_data_path = "../data/train_data.txt"
model_path = "../yttme.model"


with open(train_data_path, "w") as fout:
    fout.write(traintxt)

# Generating random text
test_text = 'привет как твои дела?'

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=30000, model=model_path)

# Loading model
bpe = yttm.BPE(model=model_path)

# Two types of tokenization
print(bpe.encode([test_text], output_type=yttm.OutputType.ID))
print(bpe.encode([test_text], output_type=yttm.OutputType.SUBWORD))

[[7017, 365, 6622, 795]]
[['▁привет', '▁как', '▁твои', '▁дела']]


In [16]:
!pwd

/Users/ibragim/Documents/punctuation-restoration/src


In [18]:
import youtokentome as yttm


bpe = yttm.BPE("../yttme.model", n_threads=-1)

In [19]:
bpe.vocab_size()

30000

In [23]:
test_text = 'привет'
print(bpe.encode(test_text, output_type=yttm.OutputType.SUBWORD))

['▁привет']


In [5]:
from prep import Preprocessor
from glob import glob

pr = Preprocessor()

In [6]:
dfs = []
for fn in glob('../data/ru/**/**/*.txt'):
    df = pr.prep_file(fn)
    dfs.append(df)

In [7]:
import pandas as pd

aldf = pd.concat(dfs).reset_index(drop=True)

In [8]:
tags=list(set(df['label'].values))
print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

['O', 'COMMA', 'QUESTION', 'PERIOD']
Number of Tags  4
Tags Map  {'O': 0, 'COMMA': 1, 'QUESTION': 2, 'PERIOD': 3}


In [10]:
reverse_tag_map={v: k for k, v in tags_map.items()}